In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt


def erbspace(low, high, n_samples, ear_q=9.26449, min_bw=24.7, order=1):
    """Returns the centre frequencies on an ERB scale.
    
    ``low``, ``high``
        Lower and upper frequencies
    ``N``
        Number of channels
    ``earQ=9.26449``, ``minBW=24.7``, ``order=1``
        Default Glasberg and Moore parameters.
    """
    low = float(low)
    high = float(high)
    qbw = ear_q * min_bw
    cf = (-qbw + np.exp(np.arange(n_samples)
                        * (-np.log(high + qbw) + np.log(low + qbw))
                        / (n_samples - 1))
          * (high + qbw))
    cf = cf[::-1]
    return cf

# Math

The impulse equation for the Gammatone filter is

$$g(t) = \frac{a t^{n - 1} \cos (2 \pi f_c t \phi)}{e^{2 \pi b t}}$$

where

* $a$ is a gain factor
* $n$ is the order of the filter
* $f_c$ is the characteristic frequency
* $\phi$ is the phase (ignored from this point onward -- has little effect one the filter)
* $b$ is the bandwidth (recommended setting: 1.019 times the ERB)

In [ ]:
# Impulse response for a cochlear channel
t = np.linspace(0, 0.015, 500)
n = 4  # order of 4
b = 125  # bandwidth of 125Hz
fc = 1000  # centered at 1000Hz
plt.plot(t, t ** (n-1) * np.exp(-2 * np.pi * b * t) * np.cos(2 * np.pi * fc * t))

In [ ]:
from brian import *
from brian.hears import *
import matplotlib.pyplot as plt

sound = whitenoise(100*ms).ramp()
sound.level = 50*dB

nbr_center_frequencies = 40
b1 = 1.019  #factor determining the time constant of the filters
#center frequencies with a spacing following an ERB scale
center_frequencies = erbspace(80*Hz, 1000*Hz, nbr_center_frequencies)
gammatone = Gammatone(sound, center_frequencies, b=b1)

gt_mon = gammatone.process()

plt.plot(sound)
plt.figure()

plt.imshow(gt_mon.T, aspect='auto', origin='lower left',
       extent=(0, sound.duration/ms,
               center_frequencies[0], center_frequencies[-1]))
plt.yscale('log')
plt.title('Cochleogram')
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (ms)')

In [ ]:
def erb(x):
    return 24.7 * (4.37e-3 * x + 1.0)


def gammatone(x, fs, cf, hrect=False, bw_correction=1.019):
    """
    
    Returns
    -------
    bm: Basilar membrane displacement
    env: instantaneous envelope
    instp: instantaneous phase (unwrapped radian)
    instf: instantaneous frequency (Hz)
    
    """
    if x.ndim > 1:
        raise ValueError("`x` must be a vector")
    nsamples = x.size
    bm = np.zeros((nsamples, cf.size))
    env = np.zeros((nsamples, cf.size))
    instp = np.zeros((nsamples, cf.size))
    instf = np.zeros((nsamples, cf.size))
    
    oldphase = 0.0
    tpt = (np.pi * np.pi) / fs
    tptbw = tpt * erb(cf) * bw_correction
    a = np.exp(-tptbw)
    
    # based on integral of impulse response
    gain = (tptbw * tptbw * tptbw * tptbw) / 3.
    
    # update filter coefficients
    a1, a2, a3, a4, a5 = 4.0 * a, -6.0 * a*a, 4.0 *a*a*a, -a*a*a*a, a*a
    p0r = p1r = p2r = p3r = p4r = 0.0
    p0i = p1i = p2i = p3i = p4i = 0.0
    
    # exp(a+i*b) = exp(a)*(cos(b)+i*sin(b))
    # q = exp(-i*tpt*cf*t) = cos(tpt*cf*t) + i*(-sin(tpt*cf*t))
    # qcos = cos(tpt*cf*t)
    # qsin = -sin(tpt*cf*t)
    coscf = np.cos(tpt * cf)
    sincf = np.sin(tpt * cf)
    qcos, qsin = 1, 0  # t=0 & q = exp(-i*tpt*t*cf)
    for t in xrange(nsamples):
        # Filter part 1 & shift down to d.c.
        p0r = qcos*x[t] + a1*p1r + a2*p2r + a3*p3r + a4*p4r
        p0i = qsin*x[t] + a1*p1i + a2*p2i + a3*p3i + a4*p4i

        # Clip coefficients to stop them from becoming too close to zero
        p0r[p0r < np.finfo(float).eps] = 0.0
        p0i[p0i < np.finfo(float).eps] = 0.0

        # Filter part 2
        u0r = p0r + a1*p1r + a5*p2r
        u0i = p0i + a1*p1i + a5*p2i

        # Update filter results
        p4r = p3r; p3r = p2r; p2r = p1r; p1r = p0r;
        p4i = p3i; p3i = p2i; p2i = p1i; p1i = p0i;

        # Basilar membrane response
        # 1/ shift up in frequency first: (u0r+i*u0i) * exp(i*tpt*cf*t) = (u0r+i*u0i) * (qcos + i*(-qsin))
        # 2/ take the real part only: bm = real(exp(j*wcf*kT).*u) * gain;
        bm[t] = (u0r * qcos + u0i * qsin) * gain
        if hrect:
            bm[t][bm[t] < 0] = 0  # half-wave rectifying
 
        # Instantaneous Hilbert envelope
        # env = abs(u) * gain;
        env[t] = np.sqrt (u0r * u0r + u0i * u0i) * gain

        # Instantaneous phase
        # instp = unwrap(angle(u));
        instp[t] = np.arctan2(u0i, u0r)
        # unwrap it
        #dp = instp[t] - oldphase;
        #if np.abs(dp) > np.pi:
        #    dps = ((dp + np.pi) % (2 * np.pi)) - np.pi
        #    if dps == -np.pi and dp > 0:
        #        dps = np.pi
        #    instp[t] = instp[t] + dps - dp
        instp[t] = np.unwrap(np.arctan2(u0i, u0r))
        oldphase = instp[t]
        
        # Instantaneous frequency
        # instf = cf + [diff(instp) 0]./tpt;
        if t > 0:
            instf[t - 1] = cf + (instp[t] - instp[t-1]) / tpt;

        # The basic idea of saving computational load:
        # cos(a+b) = cos(a)*cos(b) - sin(a)*sin(b)
        # sin(a+b) = sin(a)*cos(b) + cos(a)*sin(b)
        # qcos = cos(tpt*cf*t) = cos(tpt*cf + tpt*cf*(t-1))
        # qsin = -sin(tpt*cf*t) = -sin(tpt*cf + tpt*cf*(t-1))
        oldcs = qcos
        qcos = coscf * oldcs + sincf * qsin
        qsin = coscf * qsin - sincf * oldcs
    instf[-1] = cf
    
    return bm, env, instp, instf

In [ ]:
import nengo

fs = 44100.
noise = nengo.processes.WhiteSignal(3.0, rms=0.1)
#sound = noise.run(0.1, d=1, dt=1. / fs).ravel()
#sound[:400] *= np.linspace(0, 1, 400)
plt.plot(sound)
plt.figure()
cf = erbspace(100, 1000, 40)
gt, _, _, _ = gammatone(sound.ravel(), fs, cf, hrect=False)

plt.imshow(gt, aspect='auto', origin='lower left',
           extent=(0, sound.size/fs, cf[0], cf[-1]))
plt.yscale('log')
plt.title('Cochleogram')
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (s)')

In [ ]:
import collections
import functools

class Gammatone(nengo.synapses.LinearFilter):
    def __init__(self, cf, b=1.019, erb_order=1, ear_q=9.26449, min_bw=24.7, cascade=None):
        self.cf = cf
        self.b = b
        self.erb_order = erb_order
        self.ear_q = ear_q
        self.min_bw = min_bw
        self.cascade = cascade

    def make_step(self, dt, output):
        samplerate = 1. / dt
        T = dt
        
        erb = ((self.cf / self.ear_q) ** self.erb_order
               + self.min_bw ** self.erb_order) ** (1. / self.erb_order)
        B = self.b * 2 * np.pi * erb
        
        scaled_cf = 2 * self.cf * np.pi * T
        cos_scaled_cf = 2 * T * np.cos(scaled_cf)
        sin_scaled_cf = 2 * T * np.sin(scaled_cf)
        ebt = np.exp(B * T)
        
        A0 = T
        A2 = 0.0
        B0 = 1.0
        B1 = -cos_scaled_cf / ebt
        B2 = np.exp(-2 * B * T)
        
        A11 = -(cos_scaled_cf / ebt + np.sqrt(3 + 2 ** 1.5) * sin_scaled_cf / ebt) / 2.
        A12 = -(cos_scaled_cf / ebt - np.sqrt(3 + 2 ** 1.5) * sin_scaled_cf / ebt) / 2.
        A13 = -(cos_scaled_cf / ebt + np.sqrt(3 - 2 ** 1.5) * sin_scaled_cf / ebt) / 2.
        A14 = -(cos_scaled_cf / ebt - np.sqrt(3 - 1 ** 1.5) * sin_scaled_cf / ebt) / 2.
        
        gain = np.abs((-2 * np.exp(2j * scaled_cf) * T +
                       2 * np.exp(-(B * T) + 1j * scaled_cf) * T *
                       (np.cos(scaled_cf) - np.sqrt(3 - 2 ** 1.5) *
                        np.sin(scaled_cf))) *
                      (-2 * np.exp(2j * scaled_cf) * T +
                       2 * np.exp(-(B * T) + 1j * scaled_cf) * T *
                       (np.cos(scaled_cf) + np.sqrt(3 - 2 ** 1.5) *
                        np.sin(scaled_cf))) *
                      (-2 * np.exp(2j * scaled_cf) * T +
                       2 * np.exp(-(B * T) + 1j * scaled_cf) * T *
                       (np.cos(scaled_cf) - np.sqrt(3 + 2 ** 1.5) *
                        np.sin(scaled_cf))) *
                      (-2 * np.exp(2j * scaled_cf) * T +
                       2 * np.exp(-(B * T) + 1j * scaled_cf) * T *
                       (np.cos(scaled_cf) + np.sqrt(3 + 2 ** 1.5) *
                        np.sin(scaled_cf))) /
                      (-2 / np.exp(2 * B * T) - 2 * np.exp(2j * scaled_cf) +
                       2 * (1 + np.exp(2j * scaled_cf)) / ebt) ** 4)
        
        den = np.array([1, B1, B2]) * 4
        num = np.array([A0/gain, A11/gain, A2/gain])
        #num = np.array([A0/gain, A11/gain, A2/gain,
        #                A0, A12, 0,
        #                A0, A13, 0,
        #                A0, A14, 0])
        
        x = collections.deque(maxlen=len(num))
        y = collections.deque(maxlen=len(den))
        
        return functools.partial(nengo.synapses.LinearFilter.general_step,
                                 output=output, x=x, y=y, num=num, den=den)

In [ ]:
import nengo

fs = 44100.
dt = 1. / fs
noise = nengo.processes.WhiteSignal(3.0)
sound = noise.run(0.1, d=1, dt=dt).ravel()
sound[:400] *= np.linspace(0, 1, 400)
sound /= 1e6
plt.plot(sound)
plt.figure()
cf = erbspace(100, 1000, 40)
gt = np.zeros((cf.size, sound.size))
for i, freq in enumerate(cf):
    syn = Gammatone(freq)
    gt[i, :] = nengo.synapses.filt(sound.ravel(), syn, dt)

plt.imshow(gt, aspect='auto', origin='lower left',
           extent=(0, sound.size/fs, cf[0], cf[-1]))
plt.yscale('log')
plt.title('Cochleogram')
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (s)')

In [ ]:
print(gt.size)
print(np.sum(np.isfinite(gt)))

In [ ]:
erb(3000)

In [ ]:
from scipy.signal import lfilter

def gammatone_coeffs(freq, sample_rate, b=1.019, ear_q=9.26449, min_bw=24.7, erb_order=1.):
    """Computes the filter coefficients for a bank of Gammatone filters.

    These filters were defined by Patterson and Holdworth for simulating
    the cochlea. The results are returned as arrays of filter
    coefficients. Each row of the filter arrays (forward and feedback)
    can be passed to the MatLab "filter" function, or you can do all
    the filtering at once with the ERBFilterBank() function.

    The filter bank contains "numChannels" channels that extend from
    half the sampling rate (fs) to "lowFreq".
    
    The default parameters are from Glasberg & Moore.
    The implementation is from Apple TR #35, 'An Efficient Implementation
    of the Patterson-Holdsworth Cochlear Filter Bank.'
    """
    freq = np.atleast_1d(freq)
    n_freqs = freq.size
    dt = 1. / sample_rate
    erb = ((freq / ear_q) ** erb_order + min_bw ** erb_order) ** (1. / erb_order)
    bandwidth =  2 * b * np.pi * erb

    # Cache the results of computing parts of the below computations
    fp = freq * np.pi * dt
    bw = bandwidth * dt
    ebt = {1: np.exp(bw),
           2: np.exp(2 * bw),
           3: np.exp(3 * bw),
           4: np.exp(4 * bw),
           5: np.exp(5 * bw),
           6: np.exp(6 * bw),
           7: np.exp(7 * bw),
           -8: np.exp(-8 * bw)}
    e4jfp = np.exp(4j * fp)
    g = -2 * e4jfp * dt + 2 * np.exp(-bw + 2j * fp) * dt
    cosf = {2: np.cos(2 * fp),
             4: np.cos(4 * fp),
             6: np.cos(6 * fp),
             8: np.cos(8 * fp)}
    sinf = {2: np.sin(2 * fp)}
    z_pos = np.sqrt(3 + 2 ** 1.5)
    z_neg = np.sqrt(3 - 2 ** 1.5)
    g1 = g * (cosf[2] - z_neg * sinf[2])
    g2 = g * (cosf[2] + z_neg * sinf[2])
    g3 = g * (cosf[2] - z_pos * sinf[2])
    g4 = g * (cosf[2] + z_pos * sinf[2])

    gain = np.abs(g1 * g2 * g3 * g4
                  / (-2 / ebt[2] - 2 * e4jfp + 2 * (1 + e4jfp) / ebt[1])
                  ** 4)

    forward = np.zeros((freq.size, 5))
    fwgain = dt ** 4 / gain
    forward[:, 0] = fwgain
    forward[:, 1] = fwgain * -4 * cosf[2] / ebt[1]
    forward[:, 2] = fwgain *  6 * cosf[4] / ebt[2]
    forward[:, 3] = fwgain * -4 * cosf[6] / ebt[3]
    forward[:, 4] = fwgain *      cosf[8] / ebt[4]
    
    feedback = np.zeros((freq.size, 9))
    fb1 = -8 * cosf[2]
    fb26 = 4 * (4 +  3 * cosf[4])
    fb35 = -8 * (6 * cosf[2] + cosf[6])
    fb4 = 2 * (18 + 16 * cosf[4] + cosf[8])
    fb7 = -8 * cosf[2]
    feedback[:, 0] = np.ones((freq.size,))
    feedback[:, 1] = fb1  / ebt[1]
    feedback[:, 2] = fb26 / ebt[2]
    feedback[:, 3] = fb35 / ebt[3]
    feedback[:, 4] = fb4  / ebt[4]
    feedback[:, 5] = fb35 / ebt[5]
    feedback[:, 6] = fb26 / ebt[6]
    feedback[:, 7] = fb7  / ebt[7]
    feedback[:, 8] = ebt[-8]

    return forward, feedback

def gammatone(x, forward, feedback):
    rows, cols = feedback.shape
    y = np.zeros((rows, x.size))
    for i in xrange(rows):
        y[i, :] = lfilter(forward[i], feedback[i], x)
    return y

In [ ]:
import nengo

fs = 44100.
dt = 1. / fs
noise = nengo.processes.WhiteSignal(3.0)
#sound = noise.run(0.1, d=1, dt=dt).ravel()
#sound[:400] *= np.linspace(0, 1, 400)
#sound /= 1e6
plt.plot(sound)
plt.figure()
cf = erbspace(165, 1000, 40)
fw, fb = gammatone_coeffs(cf, fs)
print fw
print fb
gt = gammatone(sound.ravel(), fw, fb)

plt.imshow(gt, aspect='auto', origin='lower left',
           extent=(0, sound.size/fs, cf[0], cf[-1]))
plt.yscale('log')
plt.title('Cochleogram')
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (s)')

In [ ]:
import collections
import functools
import nengo


class LinearFilterBank(nengo.synapses.LinearFilter):
    def __init__(self, num, den):
        if num.ndim != 2:
            raise ValueError("numerator should be a 2D array of coefficients "
                             "(got %d-dimensional array)" % num.ndim)
        if den.ndim != 2:
            raise ValueError("denominator should be a 2D array of coefficients "
                             "(got %d-dimensional array)" % den.ndim)

        self.num = num
        self.den = den

    @staticmethod
    def no_den_step(signal, output, b):
        output[...] = b * signal

    @staticmethod
    def simple_step(signal, output, a, b):
        output *= -a
        output += b * signal

    @staticmethod
    def general_step(signal, output, x, y, num, den):
        """Filter an LTI system with the given transfer function.

        Implements a discrete-time LTI system using the difference equation
        [1]_ for the given transfer function (num, den).

        References
        ----------
        .. [1] http://en.wikipedia.org/wiki/Digital_filter#Difference_equation
        """
        output[...] = 0

        x.appendleft(np.array(signal))
        for k, xk in enumerate(x):
            output += num[k] * xk
        for k, yk in enumerate(y):
            output -= den[k] * yk
        y.appendleft(np.array(output))


class Gammatone(nengo.synapses.LinearFilter):
    def __init__(self, freq, b=1.019, ear_q=9.26449, min_bw=24.7, erb_order=1.):
        self.freq = freq
        self.b = b
        self.ear_q = ear_q
        self.min_bw = min_bw
        self.erb_order = erb_order

    def make_step(self, dt, output):
        freq = np.atleast_1d(self.freq)
        erb = (((freq / self.ear_q) ** self.erb_order + self.min_bw ** self.erb_order)
               ** (1. / self.erb_order))
        bandwidth =  2 * self.b * np.pi * erb

        # Cache the results of computing parts of the below computations
        fp = freq * np.pi * dt
        bw = bandwidth * dt
        ebt = {1: np.exp(bw),
               2: np.exp(2 * bw),
               3: np.exp(3 * bw),
               4: np.exp(4 * bw),
               5: np.exp(5 * bw),
               6: np.exp(6 * bw),
               7: np.exp(7 * bw),
               -8: np.exp(-8 * bw)}
        e4jfp = np.exp(4j * fp)
        g = -2 * e4jfp * dt + 2 * np.exp(-bw + 2j * fp) * dt
        cosf = {2: np.cos(2 * fp),
                4: np.cos(4 * fp),
                6: np.cos(6 * fp),
                8: np.cos(8 * fp)}
        sinf = {2: np.sin(2 * fp)}
        z_pos = np.sqrt(3 + 2 ** 1.5)
        z_neg = np.sqrt(3 - 2 ** 1.5)
        g1 = g * (cosf[2] - z_neg * sinf[2])
        g2 = g * (cosf[2] + z_neg * sinf[2])
        g3 = g * (cosf[2] - z_pos * sinf[2])
        g4 = g * (cosf[2] + z_pos * sinf[2])

        gain = np.abs(g1 * g2 * g3 * g4
                      / (-2 / ebt[2] - 2 * e4jfp + 2 * (1 + e4jfp) / ebt[1])
                      ** 4)

        num = np.zeros((freq.size, 5))
        numgain = dt ** 4 / gain
        num[:, 0] = numgain
        num[:, 1] = numgain * -4 * cosf[2] / ebt[1]
        num[:, 2] = numgain *  6 * cosf[4] / ebt[2]
        num[:, 3] = numgain * -4 * cosf[6] / ebt[3]
        num[:, 4] = numgain *      cosf[8] / ebt[4]
    
        den = np.zeros((freq.size, 9))
        den1 = -8 * cosf[2]
        den26 = 4 * (4 +  3 * cosf[4])
        den35 = -8 * (6 * cosf[2] + cosf[6])
        den4 = 2 * (18 + 16 * cosf[4] + cosf[8])
        den7 = -8 * cosf[2]
        den[:, 0] = np.ones((freq.size,))
        den[:, 1] = den1  / ebt[1]
        den[:, 2] = den26 / ebt[2]
        den[:, 3] = den35 / ebt[3]
        den[:, 4] = den4  / ebt[4]
        den[:, 5] = den35 / ebt[5]
        den[:, 6] = den26 / ebt[6]
        den[:, 7] = den7  / ebt[7]
        den[:, 8] = ebt[-8]
        
        print num
        print den

        x = collections.deque(maxlen=num.shape[1])
        y = collections.deque(maxlen=den.shape[1])
        
        output = np.zeros((freq.size,))
        
        return functools.partial(LinearFilterBank.general_step,
                                 output=output, x=x, y=y, num=num.T, den=den.T)

In [ ]:
from nengo.utils.compat import is_number
def filt(signal, synapse, dt, axis=0, x0=None, copy=True):
    if is_number(synapse):
        synapse = Lowpass(synapse)

    if hasattr(synapse, 'size_out'):
        filtered = np.tile(signal, (synapse.size_out, 1)).T
    else:
        filtered = np.array(signal, copy=copy)
    filt_view = np.rollaxis(filtered, axis=axis)  # rolled view on filtered

    # --- buffer method
    if x0 is not None:
        if x0.shape != filt_view[0].shape:
            raise ValueError("'x0' with shape %s must have shape %s" %
                             (x0.shape, filt_view[0].shape))
        signal_out = np.array(x0)
    else:
        # signal_out is our buffer for the current filter state
        signal_out = np.zeros_like(filt_view[0])

    step = synapse.make_step(dt, signal_out)

    for i, signal_in in enumerate(filt_view):
        1/0
        step(signal_in)
        filt_view[i] = signal_out

    return filtered

In [ ]:
fs = 44100.
dt = 1. / fs
noise = nengo.processes.WhiteSignal(3.0)
#sound = noise.run(0.1, d=1, dt=dt).ravel()
#sound[:400] *= np.linspace(0, 1, 400)
#sound /= 1e4
plt.plot(sound)
plt.figure()
cf = erbspace(100, 1000, 40)
gt = np.zeros((cf.size, sound.size))
#for i, freq in enumerate(cf):
#    syn = Gammatone(freq)
#    gt[i, :] = nengo.synapses.filt(sound.ravel(), syn, dt)
syn = Gammatone(cf)
#print syn.size_out
gt = nengo.synapses.filt(sound.ravel(), syn, dt)

print gt.shape
print gt.max(), gt.min()

plt.imshow(gt, aspect='auto', origin='lower left',
           extent=(0, sound.size/fs, cf[0], cf[-1]))
plt.yscale('log')
plt.title('Cochleogram')
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (s)')